In [1]:
import xarray as xr
import numpy as np
import dask.array

In [2]:
temp = xr.DataArray(
    dask.array.random.normal(size=(360, 180, 12), chunks=(30, 45, 3)),
    dims=("lon", "lat", "time")
)
temp

<xarray.DataArray 'normal-2c7f3b9ff537d80f62bd4976e5399240' (lon: 360, lat: 180, time: 12)>
dask.array<normal, shape=(360, 180, 12), dtype=float64, chunksize=(30, 45, 3), chunktype=numpy.ndarray>
Dimensions without coordinates: lon, lat, time

In [3]:
mask = (temp.isel(time=0) > 0)
mask

<xarray.DataArray 'normal-2c7f3b9ff537d80f62bd4976e5399240' (lon: 360, lat: 180)>
dask.array<gt, shape=(360, 180), dtype=bool, chunksize=(30, 45), chunktype=numpy.ndarray>
Dimensions without coordinates: lon, lat

In [4]:
temp_masked = temp.where(mask)
temp_masked

<xarray.DataArray 'normal-2c7f3b9ff537d80f62bd4976e5399240' (lon: 360, lat: 180, time: 12)>
dask.array<where, shape=(360, 180, 12), dtype=float64, chunksize=(30, 45, 3), chunktype=numpy.ndarray>
Dimensions without coordinates: lon, lat, time

In [5]:
temp_masked_sub = temp_masked.isel(
    lon=slice(55, 235),
    lat=slice(67, 98),
    time=slice(3, 9)
)

temp_masked_sub

<xarray.DataArray 'normal-2c7f3b9ff537d80f62bd4976e5399240' (lon: 180, lat: 31, time: 6)>
dask.array<getitem, shape=(180, 31, 6), dtype=float64, chunksize=(30, 23, 3), chunktype=numpy.ndarray>
Dimensions without coordinates: lon, lat, time

In [6]:
from dask.distributed import Client

In [8]:
client = Client(ip="0.0.0.0")
client

/srv/conda/envs/notebook/lib/python3.7/site-packages/distributed/node.py:155: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 42123 instead
  http_address["port"], self.http_server.port


Client Scheduler: tcp://172.17.0.2:33581 Dashboard: /proxy/42123/status,Cluster Workers: 2 Cores: 2 Memory: 8.00 GB


In [15]:
grouped_quantiles = temp_masked_sub.chunk({"lat": -1}).groupby("lon").quantile(0.9, "lat").persist()
grouped_quantiles

,Array,Chunk
Bytes,8.64 kB,24 B
Shape,"(180, 6)","(1, 3)"
Count,360 Tasks,360 Chunks
Type,float64,numpy.ndarray
